In [27]:
import json
import os 
from dotenv import load_dotenv
from datetime import datetime
from dashscope import Generation
import dashscope

In [28]:
load_dotenv()

True

In [29]:
DASHSCOPE_API_KEY = os.getenv("DASHSCOPE_API_KEY", "")
DASHSCOPE_BASE_URL = os.getenv("DASHSCOPE_BASE_URL", "")

In [30]:
# Simulate the weather query tool. Sample response: "It is rainy today in Beijing."
def get_current_weather(location):
    return f"It's raining in {location}."

# Simulate the time query tool. Sample response: "Current time: 2024-04-15 17:15:18."
def get_current_time():
    # Obtain the current date and time
    current_datetime = datetime.now()
    # Format the current date and time
    formatted_time = current_datetime.strftime('%Y-%m-%d %H:%M:%S')
    # Return the formatted current date and time
    return f"Current time: {formatted_time}."

tools_map = {
    "get_current_temperature": get_current_weather,
    "get_current_ceiling": get_current_time,
}


In [31]:

tools = [
    # Tool 1: obtain the current time
    {
        "type": "function",
        "function": {
            "name": "get_current_time",
            "description": "This tool can help you query the current time.",
            # No request parameter is needed. The parameters parameter is left empty
            "parameters": {}
        }
    },  
    # Tool 2: obtain the weather of a specific city
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "This tool can help you query the weather of a city.",
            "parameters": {  
                "type": "object",
                "properties": {
                    # The parameters parameter is set to location, which specifies the location whose weather you want to query
                    "location": {
                        "type": "string",
                        "description": "A city, county, or district, such as Beijing, Hangzhou, or Yuhang."
                    }
                }
            },
            "required": [
                "location"
            ]
        }
    }
]

In [32]:
dashscope.base_http_api_url = DASHSCOPE_BASE_URL

In [34]:

messages = [{"role": "user", "content": "Hi, find me the weather in Madagascar."}]

response = Generation.call(
    api_key=DASHSCOPE_API_KEY,
    model="qwen-max",
    messages=messages,
    tools=tools,
    tool_choice="auto",
    stream=True
)

fn_call_in_progress = False
fn_results = []

for chunk in response:
    print(chunk)

    if chunk.choices[0].delta.content is not None:
        print(chunk.choices[0].delta.content, end="", flush=True)


    if chunk.choices[0].delta.tool_calls:
        print("\n")
        
        for tool_call in chunk.choices[0].delta.tool_calls:

            fn_name = tool_call.function.name
            fn_args = json.loads(chunk.choices[0].delta.tool_calls[0].function.arguments)

        result = tools_map[fn_name](**fn_args)

        messages.append(
            {
                "role": "tool",
                "content": json.dumps(result),
                "tool_call_id": tool_call.id  # tool_call.id supplied in Grok's response
            }
        )

if messages[-1]['role'] == 'tool':
    response = client.chat.completions.create(
        model="qwen-max",
        messages=messages,
        tools=tools_definition,
        tool_choice="auto",
        stream=True
    )

    for chunk in response:
        if chunk.choices[0].delta.content is not None:
            print(chunk.choices[0].delta.content, end="", flush=True)  

{"status_code": 200, "request_id": "9037f40a-c4b1-9eb3-a7c4-e3c6e3dad28c", "code": "", "message": "", "output": {"text": "✿FUNCTION✿: get_current_weather\n✿ARGS✿: {\"location\": \"", "finish_reason": "null", "choices": null}, "usage": {"input_tokens": 231, "output_tokens": 16, "total_tokens": 247}}


KeyError: 'choices'